In [1]:
import requests
import pandas as pd
import defs
import os

/Users/egrados/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle("instruments.pkl")

In [4]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [5]:
def fetch_candles (pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json() 

#code, res = fetch_candles("EUR_USD", 10, "H1")

In [6]:
def get_candles_df(json_response):
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]
    our_data = []

    for candle in json_response["candles"]:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)
    


In [7]:
#code, res = fetch_candles("EUR_USD", 10, "H1")
#df = get_candles_df(res)

In [8]:
def save_file(candles_df, pair, granularity):
    path = os.path.join("his_data", f"{pair}_{granularity}.pkl")
    candles_df.to_pickle(path)


In [9]:
def create_data(pair, granularity): #4000 datos por llamada
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, "ERROR")
        return 
    df = get_candles_df(json_data)
    print (f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)
    


In [10]:
# Itera sobre todas las comnbinaciones posibles de los currencys que queremos  tradear, no sobre todos los pares posibles de OANDA
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")
        

EUR_USD loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2023-11-06T17:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
USD_JPY loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
USD_CHF loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
USD_CAD loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
GBP_USD loaded 3999 candles from 2023-11-06T15:00:00.000000000Z to 2024-06-28T06:00:00.000000000Z
GBP_JPY loaded 3999 